<a href="https://colab.research.google.com/github/sazio/MultiAgentCovid/blob/master/Multiplex_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Data and match Agents with Buildings

In [132]:
import numpy as np
import pandas as pd 
import glob 
from tqdm.notebook import tqdm 
from scipy import sparse

In [81]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [82]:
path = "drive/MyDrive/MAS/Data/Output_GAMA/Data"

agents_path = path + "/Agents"
buildings_path = path + "/Buildings"

agents_files = glob.glob(agents_path + "/*.csv")
buildings_file = glob.glob(buildings_path + "/*.csv")

agents_1_day = [agents_files[i] for i in range(0,24)]

In [5]:
DF_buildings = pd.read_csv(buildings_file[0], sep = ",", names = ["name", "loc_x", "loc_y", "loc_z"])
DF_buildings = DF_buildings.iloc[1:]
DF_buildings = DF_buildings.reset_index(drop = True)
DF_buildings["name"] = DF_buildings["name"].str[:8] + "(" + DF_buildings["name"].str[8:] + ")"
DF_buildings["loc_x"] = DF_buildings["loc_x"].str[1:].astype(np.float32)
DF_buildings["loc_y"] = DF_buildings["loc_y"].astype(np.float32)
DF_buildings["loc_z"] = DF_buildings["loc_z"].str[:-1].astype(np.float32)

DF_buildings.head()

,name,loc_x,loc_y,loc_z
0,building(0),4258.587891,8622.923828,0.0
1,building(1),4205.688477,8554.613281,0.0
2,building(2),4172.957031,8508.589844,0.0
3,building(3),4653.243164,8230.745117,0.0
4,building(4),4630.689941,8173.111816,0.0


In [6]:
# First Agents file as example (ToDo --> function from this stuff)
DF_agents = pd.read_csv(agents_files[0], sep = ",", names = ["name", "cycle","loc_x", "loc_y", "loc_z","living_place",
"working_place","stuff_place", "is_susceptible", "is_infected", "is_immune"])
DF_agents = DF_agents.iloc[1:]
DF_agents = DF_agents.reset_index(drop = True)
DF_agents["loc_x"] = DF_agents["loc_x"].str[1:].astype(np.float32)
DF_agents["loc_y"] = DF_agents["loc_y"].astype(np.float32)
DF_agents["loc_z"] = DF_agents["loc_z"].str[:-1].astype(np.float32)

DF_agents = DF_agents.replace({"is_susceptible":{"true":True, "false":False}})
DF_agents = DF_agents.replace({"is_infected":{"True":True, "False":False}})
DF_agents = DF_agents.replace({"is_immune":{"True":True, "False":False}})

DF_agents.head()

,name,cycle,loc_x,loc_y,loc_z,living_place,working_place,stuff_place,is_susceptible,is_infected,is_immune
0,people0,401,4047.333740,2014.397461,0.0,building(1408),building(220),building(3381),False,False,True
1,people1,401,1911.168823,3917.855225,0.0,building(1350),building(5862),building(6186),False,True,False
2,people2,401,3277.526123,657.434082,0.0,building(2112),building(84),building(6398),True,False,False
3,people3,401,3803.600098,1790.961182,0.0,building(5496),building(5860),building(6211),True,False,False
4,people4,401,3402.263672,7576.838379,0.0,building(1101),building(266),building(2286),True,False,False


In [84]:
def is_in_which_building(DF_buildings, DF_agents, agent_index):

  """
  Check if the agent is in some building or in the street (if alive), useful to build a multiplex network 

  """
  # In case some of the locations are "nil", i.e. non existing, it basically means that our agent is dead
  if DF_agents.values[agent_index,5]  == "nil":
    return "Dead"

  elif DF_agents.values[agent_index,6] == "nil":  
    return "Dead"

  elif DF_agents.values[agent_index,7] == "nil": 
    return "Dead"

  # In case our agent is alive 
  else:

    living_coord = DF_buildings[DF_buildings.name == DF_agents.values[agent_index,5]].values[0][1:3]
    working_coord = DF_buildings[DF_buildings.name == DF_agents.values[agent_index,6]].values[0][1:3]
    stuff_coord = DF_buildings[DF_buildings.name == DF_agents.values[agent_index,7]].values[0][1:3]

    agent_coord = DF_agents.values[agent_index,2:4]

    liv_ag_dist = np.linalg.norm(living_coord - agent_coord)
    wor_ag_dist = np.linalg.norm(working_coord - agent_coord)
    stu_ag_dist = np.linalg.norm(stuff_coord - agent_coord)

    if min(liv_ag_dist, wor_ag_dist, stu_ag_dist) == liv_ag_dist and liv_ag_dist < 50:

      return DF_agents.values[agent_index,5]

    elif  min(liv_ag_dist, wor_ag_dist, stu_ag_dist) == wor_ag_dist and wor_ag_dist < 50:

      return DF_agents.values[agent_index,6]

    elif  min(liv_ag_dist, wor_ag_dist, stu_ag_dist) == stu_ag_dist and stu_ag_dist < 50:

      return DF_agents.values[agent_index,7]

    # In case our agent is on the road to some place 
    else: 

      return "None"

In [85]:
# ToDo, iterrows is not the most efficient, find a faster way (e.g. apply)
for index,row in tqdm(DF_agents.iterrows(), total = DF_agents.shape[0]):
    DF_agents.loc[index,'current_location'] = is_in_which_building(DF_buildings, DF_agents, agent_index = index)

In [86]:
DF_agents[DF_agents.current_location.duplicated()].head()

,name,cycle,loc_x,loc_y,loc_z,living_place,working_place,stuff_place,is_susceptible,is_infected,is_immune,current_location
6,people6,401,2042.102539,3271.883545,0.0,building(6504),building(5664),building(1325),False,False,True,None
10,people10,401,3804.095215,1785.334595,0.0,building(1800),building(336),building(6211),True,False,False,building(6211)
13,people15,401,3664.964355,2534.839355,0.0,building(2221),building(260),building(6446),True,False,False,None
14,people16,401,4290.428223,4583.939941,0.0,building(6702),building(1228),building(6186),False,False,True,None
18,people20,401,3480.477539,395.809479,0.0,building(5497),building(222),building(1325),False,False,True,None


In [87]:
DF_agents[DF_agents.current_location == "building(6211)"].head()

,name,cycle,loc_x,loc_y,loc_z,living_place,working_place,stuff_place,is_susceptible,is_infected,is_immune,current_location
3,people3,401,3803.600098,1790.961182,0.0,building(5496),building(5860),building(6211),True,False,False,building(6211)
10,people10,401,3804.095215,1785.334595,0.0,building(1800),building(336),building(6211),True,False,False,building(6211)
62,people69,401,3817.859375,1793.564697,0.0,building(904),building(2088),building(6211),False,False,True,building(6211)
90,people101,401,3826.621826,1795.730347,0.0,building(1330),building(306),building(6211),True,False,False,building(6211)
100,people117,401,3827.175537,1791.492920,0.0,building(1418),building(2241),building(6211),False,False,True,building(6211)


# Multiplex Analysis

In [103]:
#!pip install git+https://github.com/nkoub/multinetx.git

In [89]:
import multinetx as mx

In [123]:
# Create weighted links in the same network 
# DF_agents[DF_agents.current_location == DF_agents.living_place]

In [91]:
DF_agents.head()

,name,cycle,loc_x,loc_y,loc_z,living_place,working_place,stuff_place,is_susceptible,is_infected,is_immune,current_location
0,people0,401,4047.333740,2014.397461,0.0,building(1408),building(220),building(3381),False,False,True,None
1,people1,401,1911.168823,3917.855225,0.0,building(1350),building(5862),building(6186),False,True,False,building(1350)
2,people2,401,3277.526123,657.434082,0.0,building(2112),building(84),building(6398),True,False,False,building(6398)
3,people3,401,3803.600098,1790.961182,0.0,building(5496),building(5860),building(6211),True,False,False,building(6211)
4,people4,401,3402.263672,7576.838379,0.0,building(1101),building(266),building(2286),True,False,False,building(266)


In [131]:
# Adjacency matrix for each Network 
# ToDo: make it faster by vectorization (especially for larger matrices)
living_adj = np.eye(DF_agents.shape[0])

for i in tqdm(range(0,living_adj.shape[0])):
  for j in range(0,living_adj.shape[0]):

    if DF_agents.loc[i].living_place == DF_agents.loc[j].living_place:
      living_adj[i,j] = 1  

    else: 
      living_adj[i,j] = 0 

In [133]:
sparse_living_adj = sparse.csr_matrix(living_adj)
sparse.save_npz(path + '/sparse_living_adj.npz', sparse_living_adj)

In [ ]:
#  ToDo: make it faster by vectorization 
working_adj = np.eye(DF_agents.shape[0])

for i in tqdm(range(0,working_adj.shape[0])):
  for j in range(0,working_adj.shape[0]):

    if DF_agents.loc[i].working_place == DF_agents.loc[j].working_place:
      working_adj[i,j] = 1  

    else: 
      working_adj[i,j] = 0 

In [ ]:
sparse_working_adj = sparse.csr_matrix(working_adj)
sparse.save_npz(path + '/sparse_working_adj.npz', sparse_working_adj)

In [ ]:
#  ToDo: make it faster by vectorization 
stuff_adj = np.eye(DF_agents.shape[0])

for i in tqdm(range(0,stuff_adj.shape[0])):
  for j in range(0,stuff_adj.shape[0]):

    if DF_agents.loc[i].stuff_place == DF_agents.loc[j].stuff_place:
      stuff_adj[i,j] = 1  

    else: 
      stuff_adj[i,j] = 0 

In [ ]:
sparse_stuff_adj = sparse.csr_matrix(stuff_adj)
sparse.save_npz(path + '/sparse_stuff_adj.npz', sparse_stuff_adj)

In [ ]:
# InterAdjacency matrix --> connections between places of the same agent
